# 环境配置

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz 
from sklearn import metrics
from sklearn.metrics import roc_curve, auc 
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split 
import eli5 
from eli5.sklearn import PermutationImportance

In [3]:
import shap 
from pdpbox import pdp, info_plots 
np.random.seed(2022) 

pd.options.mode.chained_assignment = None  
%matplotlib inline

In [4]:
#查看当下路径
import os
print(os.getcwd())
os.chdir("/share/home/linmiao/bio/2022_heartEC_scRNA/mRNA/230508_3Data")
print(os.getcwd())

/share/home/linmiao/bio/2022_heartEC_scRNA/mRNA/230508_3Data
/share/home/linmiao/bio/2022_heartEC_scRNA/mRNA/230508_3Data


In [5]:
# 导入数据
dc = pd.read_csv('./discovery/discovery.csv', index_col=0)
# 输出首尾五行
print(dc.head())
#查看数据集一些信息 
dc.info()

                VEGFA       FN1       EGF       KDR    PECAM1      FGF2  \
653513_mRNA  215.3208  72.10228  79.49622  81.47382  397.2655  87.55611   
647527_mRNA  233.5941  60.83722  79.64598  81.01164  280.4375  76.69513   
634021_mRNA  180.6582  63.64766  75.90014  74.75217  269.5177  68.91901   
43_mRNA      198.6208  64.54980  71.41699  83.72794  287.8640  83.03422   
5059_mRNA    199.9315  69.61796  72.64426  78.08111  677.6162  77.11231   

                ITGB1    NOTCH1     HIF1A       TEK  ...     OVOL2     VEZF1  \
653513_mRNA  1967.808  345.0295  165.5901  449.1410  ...  67.89413  1540.014   
647527_mRNA  1760.400  292.6602  129.4617  343.2925  ...  78.97446  1647.732   
634021_mRNA  1227.290  313.0690  102.2128  380.1647  ...  78.61082  1447.595   
43_mRNA      1580.264  322.4279  120.7920  374.5043  ...  62.40175  1433.261   
5059_mRNA    1715.637  371.7405  172.9951  660.7704  ...  70.67875  1502.072   

               CALCRL      EGR3      ELK3      RTN4   SLC12A6    THS

In [6]:
# 按需求导出17列
columns_to_extract = ['ARHGAP24', 'COL4A1', 'COL4A2', 'EPHB1', 'ETS1', 'FMNL3', 'PARVA', 'PLXND1', 'TGFBR2', 'VAV3',
                      'YWHAZ', 'CCL2', 'CAV1', 'PTEN', 'PTPRB', 'TEK', 'TIE1', 'group']


# Create a DataFrame
dc_17 = dc[columns_to_extract]
print(dc_17)
os.chdir("/share/home/linmiao/bio/2022_heartEC_scRNA/mRNA/230508_3Data/0512_top17")
print(os.getcwd())
# Export the DataFrame to a CSV file
dc_17.to_csv('top17_discovery.csv')

              ARHGAP24     COL4A1    COL4A2     EPHB1       ETS1     FMNL3  \
653513_mRNA   173.9474  1153.3340  130.7195  70.50249   836.2568  77.24225   
647527_mRNA   213.4493  1017.4710  123.7303  76.68093   616.1385  79.71115   
634021_mRNA   140.1534   756.1304  126.5976  83.70486   632.9750  75.00182   
43_mRNA       174.3299  1147.0020  132.4949  62.59674   822.0433  77.49987   
5059_mRNA     110.1583  2392.9340  292.8405  72.85714  1121.7880  79.84651   
...                ...        ...       ...       ...        ...       ...   
5033_mRNA     138.2048  1260.5870  176.7277  71.94365   786.8335  83.61214   
R58_mRNA      117.4417   922.0298  134.5576  66.47276  1083.6370  86.97866   
673482_mRNA   175.5019   848.8244  146.1684  67.42986   869.2896  75.95892   
R656518_mRNA  153.7022   833.1069  125.4278  70.97372   943.8604  72.10357   
R59_mRNA      218.4659  1321.5380  179.7313  72.03877  1199.1030  84.86176   

                 PARVA    PLXND1     TGFBR2      VAV3      YWHA

In [7]:
os.chdir("/share/home/linmiao/bio/2022_heartEC_scRNA/mRNA/230508_3Data/validation1")
print(os.getcwd())
# Load the external validation set 1
V1 = pd.read_csv('validation1_new.csv', index_col=0)  # Assuming the external validation set is in a CSV file named 'validation1.csv'

/share/home/linmiao/bio/2022_heartEC_scRNA/mRNA/230508_3Data/validation1


In [8]:
os.chdir("/share/home/linmiao/bio/2022_heartEC_scRNA/mRNA/230508_3Data/0512_top17")
print(os.getcwd())
# 按需求导出17列
# Create a list of the 20 values
columns_to_extract = ['ARHGAP24', 'COL4A1', 'COL4A2', 'EPHB1', 'ETS1', 'FMNL3', 'PARVA', 'PLXND1', 'TGFBR2', 'VAV3',
                      'YWHAZ', 'CCL2', 'CAV1', 'PTEN', 'PTPRB', 'TEK', 'TIE1', 'group']

# Create a DataFrame
V1_17 = V1[columns_to_extract]
print(V1_17)

# Export the DataFrame to a CSV file
V1_17.to_csv('top17_validation1.csv')

/share/home/linmiao/bio/2022_heartEC_scRNA/mRNA/230508_3Data/0512_top17
         ARHGAP24     COL4A1     COL4A2     EPHB1       ETS1      FMNL3  \
C00055  12.155062  15.533658  14.650068  6.971840  14.804589  12.868542   
C00192  12.321864  16.169583  15.639637  7.201770  13.658162  11.211754   
P01067  12.449762  17.001995  16.127677  6.639051  15.105061  13.240256   
P01101  12.831036  15.574966  15.010206  7.514270  13.514730  11.478876   
P01121  12.717077  15.654666  14.949760  7.509732  14.475517  12.318479   
P01140  12.070382  15.560914  15.554168  7.068580  13.973064  12.531941   
P01155  13.035347  17.108345  16.398600  7.365902  14.172579  12.516718   
P01184  12.127712  16.260761  15.930863  8.652672  14.252203  11.955984   
P01199  12.389707  16.068442  15.857720  7.301258  14.678115  12.899440   
P01202  11.968525  15.996370  15.584466  7.533235  14.393780  12.403795   
P01211  12.356070  14.789972  13.633743  7.285999  14.384031  12.554866   
P01226  12.821169  15.760373

In [9]:
os.chdir("/share/home/linmiao/bio/2022_heartEC_scRNA/mRNA/230508_3Data/validation2")
print(os.getcwd())
# Load the external validation set 1
V2 = pd.read_csv('validation2.csv', index_col=0)  # Assuming the external validation set is in a CSV file named 'validation1.csv'
V2.head()

os.chdir("/share/home/linmiao/bio/2022_heartEC_scRNA/mRNA/230508_3Data/0512_top17")
print(os.getcwd())
# Create a list of the 20 values
columns_to_extract = ['ARHGAP24', 'COL4A1', 'COL4A2', 'EPHB1', 'ETS1', 'FMNL3', 'PARVA', 'PLXND1', 'TGFBR2', 'VAV3',
                      'YWHAZ', 'CCL2', 'CAV1', 'PTEN', 'PTPRB', 'TEK', 'TIE1', 'group']

# Create a DataFrame
V2_17 = V2[columns_to_extract]
print(V2_17)

# Export the DataFrame to a CSV file
V2_17.to_csv('top17_validation2.csv')

/share/home/linmiao/bio/2022_heartEC_scRNA/mRNA/230508_3Data/validation2
/share/home/linmiao/bio/2022_heartEC_scRNA/mRNA/230508_3Data/0512_top17
            ARHGAP24  COL4A1  COL4A2  EPHB1  ETS1  FMNL3  PARVA  PLXND1  \
GSM4887610       204    3133    2534      6   934    259   1165     605   
GSM4887611       152    2579    2065      5   839    388    956     373   
GSM4887612       221    5067    3643     14  1204    601   1189     702   
GSM4887613       182    2293    1975      5   730    222    481     312   
GSM4887614       336    5154    4568      9  1295    390   1295    1131   
GSM4887615       210    3528    2472      8   682    344    708     521   
GSM4887616       252    3525    3094     11   883    348    666     614   
GSM4887617       193    2753    1935      8   737    271    826     351   
GSM4887618       129    2083    1877      8   487    301    360     437   
GSM4887619       187    4061    2613     12  1110    414    539     656   
GSM4887620       336    1794  

In [10]:
os.chdir("/share/home/linmiao/bio/2022_heartEC_scRNA/mRNA/230508_3Data/validation3")
print(os.getcwd())
# Load the external validation set 1
V3 = pd.read_csv('validation3.csv', index_col=0)  # Assuming the external validation set is in a CSV file named 'validation1.csv'
V3.head()

os.chdir("/share/home/linmiao/bio/2022_heartEC_scRNA/mRNA/230508_3Data/0512_top17")
print(os.getcwd())
# Create a list of the 20 values
columns_to_extract = ['ARHGAP24', 'COL4A1', 'COL4A2', 'EPHB1', 'ETS1', 'FMNL3', 'PARVA', 'PLXND1', 'TGFBR2', 'VAV3',
                      'YWHAZ', 'CCL2', 'CAV1', 'PTEN', 'PTPRB', 'TEK', 'TIE1', 'group']

# Create a DataFrame
V3_17 = V3[columns_to_extract]
print(V3_17)

# Export the DataFrame to a CSV file
V3_17.to_csv('top17_validation3.csv')

/share/home/linmiao/bio/2022_heartEC_scRNA/mRNA/230508_3Data/validation3
/share/home/linmiao/bio/2022_heartEC_scRNA/mRNA/230508_3Data/0512_top17
         ARHGAP24    COL4A1    COL4A2     EPHB1      ETS1     FMNL3     PARVA  \
HCM269    55.4523   6340.89   4926.12   5.00776   65.7462   182.714  1191.380   
HCM273    77.9603   6124.06   5612.12   5.69952  222.0810   745.710  1212.910   
HCM282   134.0970   6334.44   5248.33   4.35789  240.6990   387.203  1435.520   
HCM395   114.2030   5868.40   4618.29   9.70138  157.1520   557.049  1167.040   
HCM405   163.8900   8772.51   6987.12  20.12500  213.0870   858.220  1066.680   
HCM420   108.1850   9033.39   7959.07  10.29820  160.8780   978.668  1315.730   
HCM429   140.7840  13277.00   9943.69  13.25970  232.0440   991.443  1183.990   
HCM431   110.6740   5100.78   5038.31   3.47387  228.9590   612.283  1266.640   
HCM437    73.1783   6024.83   5844.67  10.41630  254.5340   746.388  1540.790   
HCM439   126.4170  12664.40  10032.90  19.541

# 模型训练

In [11]:
os.chdir("/share/home/linmiao/bio/2022_heartEC_scRNA/mRNA/230508_3Data/0512_top17")
print(os.getcwd())


/share/home/linmiao/bio/2022_heartEC_scRNA/mRNA/230508_3Data/0512_top17


In [12]:
# 导入数据
dc = pd.read_csv('top17_discovery.csv', index_col=0)
# 输出首尾五行
print(dc.head())
#查看数据集一些信息 
dc.info()

             ARHGAP24     COL4A1    COL4A2     EPHB1       ETS1     FMNL3  \
653513_mRNA  173.9474  1153.3340  130.7195  70.50249   836.2568  77.24225   
647527_mRNA  213.4493  1017.4710  123.7303  76.68093   616.1385  79.71115   
634021_mRNA  140.1534   756.1304  126.5976  83.70486   632.9750  75.00182   
43_mRNA      174.3299  1147.0020  132.4949  62.59674   822.0433  77.49987   
5059_mRNA    110.1583  2392.9340  292.8405  72.85714  1121.7880  79.84651   

                PARVA    PLXND1     TGFBR2      VAV3      YWHAZ      CCL2  \
653513_mRNA  136.4923  50.83784   939.6944  78.56113   965.0801  138.3245   
647527_mRNA  136.6560  51.28878   774.0513  75.42979  1081.1250  158.4685   
634021_mRNA  101.2528  59.64109   653.4506  71.55980   926.0781  204.7034   
43_mRNA      116.3183  51.88396   652.5029  73.09205   870.2639  267.5554   
5059_mRNA    178.6028  49.42131  1382.0050  74.25615  1416.4040  460.4003   

                 CAV1      PTEN     PTPRB       TEK       TIE1  group  
65

In [13]:
# 从dc中提取变量特征和目标变量
X = np.log(dc.drop('group', axis=1))
print(X.head())
Y = dc['group']

# 将数据切分成训练集和测试集
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=1234,shuffle=True, stratify=None)

             ARHGAP24    COL4A1    COL4A2     EPHB1      ETS1     FMNL3  \
653513_mRNA  5.158753  7.050412  4.873054  4.255648  6.728936  4.346947   
647527_mRNA  5.363399  6.925075  4.818104  4.339653  6.423472  4.378409   
634021_mRNA  4.942738  6.628214  4.841014  4.427297  6.450431  4.317512   
43_mRNA      5.160949  7.044907  4.886544  4.136713  6.711793  4.350276   
5059_mRNA    4.701918  7.780276  5.679628  4.288501  7.022679  4.380106   

                PARVA    PLXND1    TGFBR2      VAV3     YWHAZ      CCL2  \
653513_mRNA  4.916268  3.928641  6.845555  4.363877  6.872211  4.929602   
647527_mRNA  4.917467  3.937472  6.651638  4.323202  6.985757  5.065556   
634021_mRNA  4.617620  4.088345  6.482267  4.270533  6.830959  5.321562   
43_mRNA      4.756330  3.949010  6.480816  4.291720  6.768796  5.589327   
5059_mRNA    5.185164  3.900382  7.231291  4.307521  7.255877  6.132096   

                 CAV1      PTEN     PTPRB       TEK      TIE1  
653513_mRNA  7.732889  5.653844  5

按11/1 Split (70/10/20)区分

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import cross_validate

# 分割数据集
X_train_val, X_test, Y_train_val, Y_test = train_test_split(X, Y, test_size=0.0909, random_state=1234, shuffle=True, stratify=None)
X_train, X_val, Y_train, Y_val = train_test_split(X_train_val, Y_train_val, test_size=0.0909, random_state=1234, shuffle=True, stratify=None)

# 定义并训练模型
catboost_model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=8, random_seed=1234)
catboost_model.fit(X_train, Y_train, eval_set=(X_val, Y_val), early_stopping_rounds=100, verbose=False)

# 对模型性能进行评价
scoring = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
cv_results = cross_validate(catboost_model, X_train_val, Y_train_val, scoring=scoring, cv=5)

# 保存评估结果为DataFrame
result = {
    'model': type(catboost_model).__name__,
    'accuracy_mean': cv_results['test_accuracy'].mean(),
    'accuracy_std': cv_results['test_accuracy'].std(),
    'precision_mean': cv_results['test_precision'].mean(),
    'precision_std': cv_results['test_precision'].std(),
    'recall_mean': cv_results['test_recall'].mean(),
    'recall_std': cv_results['test_recall'].std(),
    'f1_score_mean': cv_results['test_f1'].mean(),
    'f1_score_std': cv_results['test_f1'].std(),
    'auc_roc_mean': cv_results['test_roc_auc'].mean(),
    'auc_roc_std': cv_results['test_roc_auc'].std()
}
result_df = pd.DataFrame(result, index=[0])
# 保存评估结果为CSV文件
result_df.to_csv('evaluation_results_111_Split.csv', index=False)

0:	learn: 0.6049315	total: 17ms	remaining: 17s
1:	learn: 0.5424663	total: 20.4ms	remaining: 10.2s
2:	learn: 0.4795015	total: 22.6ms	remaining: 7.53s
3:	learn: 0.4232847	total: 24.9ms	remaining: 6.2s
4:	learn: 0.3749547	total: 27ms	remaining: 5.36s
5:	learn: 0.3398797	total: 29ms	remaining: 4.8s
6:	learn: 0.3129289	total: 31ms	remaining: 4.4s
7:	learn: 0.2893099	total: 33ms	remaining: 4.09s
8:	learn: 0.2739261	total: 35ms	remaining: 3.85s
9:	learn: 0.2546674	total: 37.2ms	remaining: 3.68s
10:	learn: 0.2432372	total: 39.4ms	remaining: 3.54s
11:	learn: 0.2194664	total: 41.4ms	remaining: 3.41s
12:	learn: 0.2022087	total: 48.3ms	remaining: 3.67s
13:	learn: 0.1853202	total: 50.3ms	remaining: 3.54s
14:	learn: 0.1742589	total: 52.5ms	remaining: 3.44s
15:	learn: 0.1648924	total: 54.8ms	remaining: 3.37s
16:	learn: 0.1546836	total: 56.9ms	remaining: 3.29s
17:	learn: 0.1472054	total: 59ms	remaining: 3.22s
18:	learn: 0.1380611	total: 61.2ms	remaining: 3.16s
19:	learn: 0.1324627	total: 63.3ms	remain

171:	learn: 0.0052135	total: 390ms	remaining: 1.88s
172:	learn: 0.0051873	total: 392ms	remaining: 1.88s
173:	learn: 0.0051443	total: 394ms	remaining: 1.87s
174:	learn: 0.0051176	total: 396ms	remaining: 1.87s
175:	learn: 0.0050638	total: 398ms	remaining: 1.86s
176:	learn: 0.0050253	total: 400ms	remaining: 1.86s
177:	learn: 0.0050062	total: 402ms	remaining: 1.86s
178:	learn: 0.0049690	total: 404ms	remaining: 1.85s
179:	learn: 0.0049194	total: 406ms	remaining: 1.85s
180:	learn: 0.0048711	total: 408ms	remaining: 1.85s
181:	learn: 0.0048414	total: 410ms	remaining: 1.84s
182:	learn: 0.0048149	total: 412ms	remaining: 1.84s
183:	learn: 0.0047970	total: 414ms	remaining: 1.84s
184:	learn: 0.0047691	total: 417ms	remaining: 1.83s
185:	learn: 0.0047483	total: 419ms	remaining: 1.83s
186:	learn: 0.0047244	total: 421ms	remaining: 1.83s
187:	learn: 0.0047074	total: 423ms	remaining: 1.83s
188:	learn: 0.0046766	total: 425ms	remaining: 1.82s
189:	learn: 0.0046621	total: 427ms	remaining: 1.82s
190:	learn: 

340:	learn: 0.0025584	total: 759ms	remaining: 1.47s
341:	learn: 0.0025509	total: 761ms	remaining: 1.46s
342:	learn: 0.0025411	total: 763ms	remaining: 1.46s
343:	learn: 0.0025314	total: 765ms	remaining: 1.46s
344:	learn: 0.0025216	total: 770ms	remaining: 1.46s
345:	learn: 0.0025216	total: 771ms	remaining: 1.46s
346:	learn: 0.0025139	total: 773ms	remaining: 1.45s
347:	learn: 0.0025047	total: 775ms	remaining: 1.45s
348:	learn: 0.0025007	total: 778ms	remaining: 1.45s
349:	learn: 0.0024915	total: 781ms	remaining: 1.45s
350:	learn: 0.0024841	total: 783ms	remaining: 1.45s
351:	learn: 0.0024800	total: 785ms	remaining: 1.45s
352:	learn: 0.0024760	total: 787ms	remaining: 1.44s
353:	learn: 0.0024694	total: 789ms	remaining: 1.44s
354:	learn: 0.0024656	total: 791ms	remaining: 1.44s
355:	learn: 0.0024573	total: 796ms	remaining: 1.44s
356:	learn: 0.0024573	total: 798ms	remaining: 1.44s
357:	learn: 0.0024489	total: 800ms	remaining: 1.43s
358:	learn: 0.0024421	total: 802ms	remaining: 1.43s
359:	learn: 

516:	learn: 0.0018242	total: 1.14s	remaining: 1.07s
517:	learn: 0.0018201	total: 1.15s	remaining: 1.07s
518:	learn: 0.0018201	total: 1.15s	remaining: 1.06s
519:	learn: 0.0018200	total: 1.15s	remaining: 1.06s
520:	learn: 0.0018167	total: 1.15s	remaining: 1.06s
521:	learn: 0.0018122	total: 1.15s	remaining: 1.06s
522:	learn: 0.0018084	total: 1.16s	remaining: 1.05s
523:	learn: 0.0018039	total: 1.16s	remaining: 1.05s
524:	learn: 0.0018039	total: 1.16s	remaining: 1.05s
525:	learn: 0.0018038	total: 1.16s	remaining: 1.05s
526:	learn: 0.0018035	total: 1.16s	remaining: 1.04s
527:	learn: 0.0018035	total: 1.17s	remaining: 1.04s
528:	learn: 0.0017995	total: 1.17s	remaining: 1.04s
529:	learn: 0.0017945	total: 1.17s	remaining: 1.04s
530:	learn: 0.0017945	total: 1.17s	remaining: 1.03s
531:	learn: 0.0017901	total: 1.17s	remaining: 1.03s
532:	learn: 0.0017901	total: 1.18s	remaining: 1.03s
533:	learn: 0.0017859	total: 1.18s	remaining: 1.03s
534:	learn: 0.0017858	total: 1.18s	remaining: 1.02s
535:	learn: 

694:	learn: 0.0015329	total: 1.52s	remaining: 669ms
695:	learn: 0.0015315	total: 1.53s	remaining: 667ms
696:	learn: 0.0015304	total: 1.53s	remaining: 665ms
697:	learn: 0.0015303	total: 1.53s	remaining: 663ms
698:	learn: 0.0015273	total: 1.53s	remaining: 660ms
699:	learn: 0.0015248	total: 1.53s	remaining: 658ms
700:	learn: 0.0015248	total: 1.54s	remaining: 656ms
701:	learn: 0.0015217	total: 1.54s	remaining: 654ms
702:	learn: 0.0015178	total: 1.54s	remaining: 651ms
703:	learn: 0.0015172	total: 1.54s	remaining: 649ms
704:	learn: 0.0015172	total: 1.54s	remaining: 647ms
705:	learn: 0.0015172	total: 1.55s	remaining: 644ms
706:	learn: 0.0015143	total: 1.55s	remaining: 642ms
707:	learn: 0.0015143	total: 1.55s	remaining: 640ms
708:	learn: 0.0015112	total: 1.55s	remaining: 638ms
709:	learn: 0.0015111	total: 1.55s	remaining: 635ms
710:	learn: 0.0015111	total: 1.56s	remaining: 633ms
711:	learn: 0.0015085	total: 1.56s	remaining: 631ms
712:	learn: 0.0015048	total: 1.56s	remaining: 628ms
713:	learn: 

...

264:	learn: 0.0036281	total: 593ms	remaining: 1.64s
265:	learn: 0.0036123	total: 595ms	remaining: 1.64s
266:	learn: 0.0036003	total: 597ms	remaining: 1.64s
267:	learn: 0.0035852	total: 599ms	remaining: 1.64s
268:	learn: 0.0035726	total: 601ms	remaining: 1.63s
269:	learn: 0.0035593	total: 603ms	remaining: 1.63s
270:	learn: 0.0035435	total: 605ms	remaining: 1.63s
271:	learn: 0.0035288	total: 607ms	remaining: 1.63s
272:	learn: 0.0035171	total: 610ms	remaining: 1.62s
273:	learn: 0.0034964	total: 612ms	remaining: 1.62s
274:	learn: 0.0034814	total: 615ms	remaining: 1.62s
275:	learn: 0.0034679	total: 617ms	remaining: 1.62s
276:	learn: 0.0034542	total: 624ms	remaining: 1.63s
277:	learn: 0.0034378	total: 626ms	remaining: 1.63s
278:	learn: 0.0034235	total: 629ms	remaining: 1.62s
279:	learn: 0.0034092	total: 631ms	remaining: 1.62s
280:	learn: 0.0033950	total: 633ms	remaining: 1.62s
281:	learn: 0.0033834	total: 635ms	remaining: 1.62s
282:	learn: 0.0033703	total: 637ms	remaining: 1.61s
283:	learn: 

441:	learn: 0.0021324	total: 971ms	remaining: 1.23s
442:	learn: 0.0021278	total: 973ms	remaining: 1.22s
443:	learn: 0.0021232	total: 976ms	remaining: 1.22s
444:	learn: 0.0021220	total: 977ms	remaining: 1.22s
445:	learn: 0.0021176	total: 979ms	remaining: 1.22s
446:	learn: 0.0021132	total: 981ms	remaining: 1.21s
447:	learn: 0.0021111	total: 985ms	remaining: 1.21s
448:	learn: 0.0021062	total: 987ms	remaining: 1.21s
449:	learn: 0.0021019	total: 989ms	remaining: 1.21s
450:	learn: 0.0021019	total: 991ms	remaining: 1.21s
451:	learn: 0.0020973	total: 993ms	remaining: 1.2s
452:	learn: 0.0020928	total: 995ms	remaining: 1.2s
453:	learn: 0.0020868	total: 997ms	remaining: 1.2s
454:	learn: 0.0020867	total: 999ms	remaining: 1.2s
455:	learn: 0.0020833	total: 1s	remaining: 1.19s
456:	learn: 0.0020778	total: 1s	remaining: 1.19s
457:	learn: 0.0020728	total: 1s	remaining: 1.19s
458:	learn: 0.0020692	total: 1.01s	remaining: 1.19s
459:	learn: 0.0020649	total: 1.01s	remaining: 1.18s
460:	learn: 0.0020602	tot

624:	learn: 0.0016720	total: 1.35s	remaining: 813ms
625:	learn: 0.0016690	total: 1.36s	remaining: 811ms
626:	learn: 0.0016660	total: 1.36s	remaining: 809ms
627:	learn: 0.0016633	total: 1.36s	remaining: 807ms
628:	learn: 0.0016632	total: 1.36s	remaining: 804ms
629:	learn: 0.0016632	total: 1.36s	remaining: 802ms
630:	learn: 0.0016632	total: 1.37s	remaining: 800ms
631:	learn: 0.0016633	total: 1.37s	remaining: 797ms
632:	learn: 0.0016627	total: 1.37s	remaining: 795ms
633:	learn: 0.0016627	total: 1.37s	remaining: 793ms
634:	learn: 0.0016595	total: 1.38s	remaining: 791ms
635:	learn: 0.0016595	total: 1.38s	remaining: 788ms
636:	learn: 0.0016566	total: 1.38s	remaining: 786ms
637:	learn: 0.0016566	total: 1.38s	remaining: 783ms
638:	learn: 0.0016567	total: 1.38s	remaining: 781ms
639:	learn: 0.0016566	total: 1.38s	remaining: 779ms
640:	learn: 0.0016566	total: 1.39s	remaining: 776ms
641:	learn: 0.0016530	total: 1.39s	remaining: 774ms
642:	learn: 0.0016499	total: 1.39s	remaining: 772ms
643:	learn: 

806:	learn: 0.0014749	total: 1.74s	remaining: 416ms
807:	learn: 0.0014749	total: 1.74s	remaining: 414ms
808:	learn: 0.0014749	total: 1.75s	remaining: 412ms
809:	learn: 0.0014749	total: 1.75s	remaining: 410ms
810:	learn: 0.0014749	total: 1.75s	remaining: 408ms
811:	learn: 0.0014749	total: 1.75s	remaining: 406ms
812:	learn: 0.0014749	total: 1.75s	remaining: 403ms
813:	learn: 0.0014748	total: 1.75s	remaining: 401ms
814:	learn: 0.0014732	total: 1.76s	remaining: 399ms
815:	learn: 0.0014706	total: 1.76s	remaining: 397ms
816:	learn: 0.0014686	total: 1.76s	remaining: 395ms
817:	learn: 0.0014660	total: 1.76s	remaining: 392ms
818:	learn: 0.0014644	total: 1.77s	remaining: 390ms
819:	learn: 0.0014644	total: 1.77s	remaining: 388ms
820:	learn: 0.0014644	total: 1.77s	remaining: 386ms
821:	learn: 0.0014626	total: 1.77s	remaining: 384ms
822:	learn: 0.0014625	total: 1.78s	remaining: 382ms
823:	learn: 0.0014607	total: 1.78s	remaining: 380ms
824:	learn: 0.0014598	total: 1.78s	remaining: 378ms
825:	learn: 

996:	learn: 0.0013698	total: 2.13s	remaining: 6.4ms
997:	learn: 0.0013698	total: 2.13s	remaining: 4.26ms
998:	learn: 0.0013698	total: 2.13s	remaining: 2.13ms
999:	learn: 0.0013698	total: 2.13s	remaining: 0us


In [15]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
import pandas as pd

# 创建模型列表
models = [
    LogisticRegression(),
    SVC(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GaussianNB(),
    CatBoostClassifier(),
    LGBMClassifier(),
    XGBClassifier(),
    KNeighborsClassifier(),
    MLPClassifier(),
    GradientBoostingClassifier(),
    DecisionTreeClassifier()
]

# 创建评估指标列表
scoring = {
    'accuracy': 'accuracy',
    'precision': 'precision',
    'recall': 'recall',
    'f1_score': 'f1',
    'auc_roc': 'roc_auc'
}

# 执行交叉验证和模型评估
results = []
for model in models:
    cv_results = cross_validate(model, X_train, Y_train, cv=5, scoring=scoring)
    result = {
        'model': type(model).__name__,
        'accuracy_mean': cv_results['test_accuracy'].mean(),
        'accuracy_std': cv_results['test_accuracy'].std(),
        'precision_mean': cv_results['test_precision'].mean(),
        'precision_std': cv_results['test_precision'].std(),
        'recall_mean': cv_results['test_recall'].mean(),
        'recall_std': cv_results['test_recall'].std(),
        'f1_score_mean': cv_results['test_f1_score'].mean(),
        'f1_score_std': cv_results['test_f1_score'].std(),
        'auc_roc_mean': cv_results['test_auc_roc'].mean(),
        'auc_roc_std': cv_results['test_auc_roc'].std()
    }
    results.append(result)

# 将结果导出为CSV文件
results_df = pd.DataFrame(results)
results_df.to_csv('models_evaluation_111.csv', index=False)


Learning rate set to 0.003771
0:	learn: 0.6904071	total: 6.53ms	remaining: 6.52s
1:	learn: 0.6863819	total: 8.81ms	remaining: 4.39s
2:	learn: 0.6827998	total: 10.5ms	remaining: 3.5s
3:	learn: 0.6793188	total: 11.9ms	remaining: 2.96s
4:	learn: 0.6765705	total: 13.2ms	remaining: 2.63s
5:	learn: 0.6728185	total: 14.6ms	remaining: 2.42s
6:	learn: 0.6692054	total: 16.1ms	remaining: 2.28s
7:	learn: 0.6657265	total: 17.3ms	remaining: 2.15s
8:	learn: 0.6623551	total: 18.6ms	remaining: 2.05s
9:	learn: 0.6593309	total: 19.9ms	remaining: 1.97s
10:	learn: 0.6564578	total: 21.1ms	remaining: 1.9s
11:	learn: 0.6532953	total: 22.3ms	remaining: 1.83s
12:	learn: 0.6497696	total: 23.5ms	remaining: 1.78s
13:	learn: 0.6463141	total: 24.7ms	remaining: 1.74s
14:	learn: 0.6431406	total: 26.1ms	remaining: 1.71s
15:	learn: 0.6393202	total: 27.4ms	remaining: 1.68s
16:	learn: 0.6368765	total: 28.6ms	remaining: 1.66s
17:	learn: 0.6329269	total: 30ms	remaining: 1.64s
18:	learn: 0.6287890	total: 31.5ms	remaining: 1.

257:	learn: 0.2415935	total: 350ms	remaining: 1.01s
258:	learn: 0.2410947	total: 352ms	remaining: 1s
259:	learn: 0.2404994	total: 353ms	remaining: 1s
260:	learn: 0.2397433	total: 354ms	remaining: 1s
261:	learn: 0.2390767	total: 355ms	remaining: 1s
262:	learn: 0.2381616	total: 357ms	remaining: 1000ms
263:	learn: 0.2374846	total: 358ms	remaining: 998ms
264:	learn: 0.2368947	total: 359ms	remaining: 996ms
265:	learn: 0.2360815	total: 360ms	remaining: 994ms
266:	learn: 0.2353273	total: 361ms	remaining: 992ms
267:	learn: 0.2344899	total: 363ms	remaining: 990ms
268:	learn: 0.2337115	total: 364ms	remaining: 988ms
269:	learn: 0.2331886	total: 365ms	remaining: 987ms
270:	learn: 0.2327246	total: 366ms	remaining: 985ms
271:	learn: 0.2320781	total: 367ms	remaining: 983ms
272:	learn: 0.2312122	total: 369ms	remaining: 983ms
273:	learn: 0.2303273	total: 371ms	remaining: 982ms
274:	learn: 0.2295758	total: 372ms	remaining: 980ms
275:	learn: 0.2286879	total: 374ms	remaining: 982ms
276:	learn: 0.2282269	t

540:	learn: 0.1214611	total: 723ms	remaining: 614ms
541:	learn: 0.1211970	total: 725ms	remaining: 612ms
542:	learn: 0.1209490	total: 726ms	remaining: 611ms
543:	learn: 0.1206665	total: 727ms	remaining: 610ms
544:	learn: 0.1204768	total: 728ms	remaining: 608ms
545:	learn: 0.1201839	total: 730ms	remaining: 607ms
546:	learn: 0.1199365	total: 731ms	remaining: 606ms
547:	learn: 0.1196531	total: 733ms	remaining: 604ms
548:	learn: 0.1195122	total: 734ms	remaining: 603ms
549:	learn: 0.1192787	total: 735ms	remaining: 602ms
550:	learn: 0.1190355	total: 737ms	remaining: 600ms
551:	learn: 0.1188903	total: 738ms	remaining: 599ms
552:	learn: 0.1186285	total: 739ms	remaining: 598ms
553:	learn: 0.1183745	total: 741ms	remaining: 596ms
554:	learn: 0.1180755	total: 742ms	remaining: 595ms
555:	learn: 0.1177616	total: 743ms	remaining: 594ms
556:	learn: 0.1175881	total: 744ms	remaining: 592ms
557:	learn: 0.1173781	total: 746ms	remaining: 591ms
558:	learn: 0.1170774	total: 747ms	remaining: 589ms
559:	learn: 

832:	learn: 0.0724340	total: 1.1s	remaining: 220ms
833:	learn: 0.0723408	total: 1.1s	remaining: 219ms
834:	learn: 0.0722283	total: 1.1s	remaining: 218ms
835:	learn: 0.0721403	total: 1.1s	remaining: 216ms
836:	learn: 0.0719910	total: 1.1s	remaining: 215ms
837:	learn: 0.0719391	total: 1.1s	remaining: 214ms
838:	learn: 0.0717478	total: 1.11s	remaining: 212ms
839:	learn: 0.0716311	total: 1.11s	remaining: 211ms
840:	learn: 0.0715426	total: 1.11s	remaining: 210ms
841:	learn: 0.0714091	total: 1.11s	remaining: 208ms
842:	learn: 0.0712279	total: 1.11s	remaining: 207ms
843:	learn: 0.0711094	total: 1.11s	remaining: 206ms
844:	learn: 0.0710318	total: 1.11s	remaining: 204ms
845:	learn: 0.0709135	total: 1.11s	remaining: 203ms
846:	learn: 0.0708447	total: 1.12s	remaining: 202ms
847:	learn: 0.0707257	total: 1.12s	remaining: 200ms
848:	learn: 0.0706079	total: 1.12s	remaining: 199ms
849:	learn: 0.0705178	total: 1.12s	remaining: 198ms
850:	learn: 0.0704321	total: 1.12s	remaining: 196ms
851:	learn: 0.0702

73:	learn: 0.4839377	total: 100ms	remaining: 1.25s
74:	learn: 0.4820723	total: 102ms	remaining: 1.25s
75:	learn: 0.4800108	total: 103ms	remaining: 1.25s
76:	learn: 0.4782743	total: 104ms	remaining: 1.25s
77:	learn: 0.4765024	total: 105ms	remaining: 1.25s
78:	learn: 0.4742274	total: 107ms	remaining: 1.24s
79:	learn: 0.4719042	total: 108ms	remaining: 1.24s
80:	learn: 0.4700782	total: 109ms	remaining: 1.24s
81:	learn: 0.4686761	total: 110ms	remaining: 1.23s
82:	learn: 0.4664871	total: 111ms	remaining: 1.23s
83:	learn: 0.4647348	total: 113ms	remaining: 1.23s
84:	learn: 0.4631838	total: 114ms	remaining: 1.23s
85:	learn: 0.4609971	total: 115ms	remaining: 1.22s
86:	learn: 0.4585547	total: 116ms	remaining: 1.22s
87:	learn: 0.4566982	total: 117ms	remaining: 1.22s
88:	learn: 0.4549668	total: 119ms	remaining: 1.21s
89:	learn: 0.4527307	total: 120ms	remaining: 1.21s
90:	learn: 0.4509419	total: 121ms	remaining: 1.21s
91:	learn: 0.4489243	total: 122ms	remaining: 1.21s
92:	learn: 0.4468005	total: 123

...

36:	learn: 0.5665789	total: 57.9ms	remaining: 1.51s
37:	learn: 0.5632193	total: 59.6ms	remaining: 1.51s
38:	learn: 0.5601458	total: 61.1ms	remaining: 1.51s
39:	learn: 0.5568366	total: 62.4ms	remaining: 1.5s
40:	learn: 0.5540043	total: 64ms	remaining: 1.5s
41:	learn: 0.5516965	total: 65.7ms	remaining: 1.5s
42:	learn: 0.5486527	total: 67.1ms	remaining: 1.49s
43:	learn: 0.5456469	total: 68.5ms	remaining: 1.49s
44:	learn: 0.5425777	total: 69.8ms	remaining: 1.48s
45:	learn: 0.5399053	total: 71.4ms	remaining: 1.48s
46:	learn: 0.5373960	total: 73.2ms	remaining: 1.48s
47:	learn: 0.5347696	total: 74.4ms	remaining: 1.48s
48:	learn: 0.5319009	total: 75.7ms	remaining: 1.47s
49:	learn: 0.5289799	total: 77.2ms	remaining: 1.47s
50:	learn: 0.5262627	total: 78.7ms	remaining: 1.46s
51:	learn: 0.5232208	total: 80.2ms	remaining: 1.46s
52:	learn: 0.5205132	total: 81.5ms	remaining: 1.46s
53:	learn: 0.5179572	total: 82.8ms	remaining: 1.45s
54:	learn: 0.5158062	total: 84.2ms	remaining: 1.45s
55:	learn: 0.5125

285:	learn: 0.2120092	total: 413ms	remaining: 1.03s
286:	learn: 0.2114190	total: 414ms	remaining: 1.03s
287:	learn: 0.2107880	total: 415ms	remaining: 1.02s
288:	learn: 0.2104356	total: 417ms	remaining: 1.02s
289:	learn: 0.2100559	total: 418ms	remaining: 1.02s
290:	learn: 0.2094626	total: 420ms	remaining: 1.02s
291:	learn: 0.2087207	total: 421ms	remaining: 1.02s
292:	learn: 0.2081401	total: 423ms	remaining: 1.02s
293:	learn: 0.2074418	total: 424ms	remaining: 1.02s
294:	learn: 0.2068869	total: 426ms	remaining: 1.02s
295:	learn: 0.2061887	total: 428ms	remaining: 1.02s
296:	learn: 0.2058359	total: 429ms	remaining: 1.01s
297:	learn: 0.2054276	total: 430ms	remaining: 1.01s
298:	learn: 0.2049826	total: 432ms	remaining: 1.01s
299:	learn: 0.2044843	total: 433ms	remaining: 1.01s
300:	learn: 0.2036567	total: 434ms	remaining: 1.01s
301:	learn: 0.2031507	total: 436ms	remaining: 1.01s
302:	learn: 0.2025723	total: 437ms	remaining: 1s
303:	learn: 0.2017964	total: 438ms	remaining: 1s
304:	learn: 0.2011

522:	learn: 0.1158813	total: 751ms	remaining: 685ms
523:	learn: 0.1155854	total: 753ms	remaining: 684ms
524:	learn: 0.1153930	total: 754ms	remaining: 682ms
525:	learn: 0.1150003	total: 756ms	remaining: 681ms
526:	learn: 0.1147351	total: 757ms	remaining: 679ms
527:	learn: 0.1145790	total: 758ms	remaining: 678ms
528:	learn: 0.1142859	total: 760ms	remaining: 677ms
529:	learn: 0.1140517	total: 761ms	remaining: 675ms
530:	learn: 0.1138448	total: 763ms	remaining: 674ms
531:	learn: 0.1136489	total: 764ms	remaining: 672ms
532:	learn: 0.1134545	total: 765ms	remaining: 671ms
533:	learn: 0.1132816	total: 767ms	remaining: 669ms
534:	learn: 0.1131124	total: 768ms	remaining: 668ms
535:	learn: 0.1128976	total: 770ms	remaining: 666ms
536:	learn: 0.1125245	total: 771ms	remaining: 665ms
537:	learn: 0.1123561	total: 775ms	remaining: 666ms
538:	learn: 0.1121121	total: 777ms	remaining: 665ms
539:	learn: 0.1117879	total: 779ms	remaining: 663ms
540:	learn: 0.1115840	total: 780ms	remaining: 662ms
541:	learn: 

782:	learn: 0.0696141	total: 1.13s	remaining: 312ms
783:	learn: 0.0694918	total: 1.13s	remaining: 311ms
784:	learn: 0.0692989	total: 1.13s	remaining: 309ms
785:	learn: 0.0692086	total: 1.13s	remaining: 308ms
786:	learn: 0.0690317	total: 1.13s	remaining: 306ms
787:	learn: 0.0689429	total: 1.13s	remaining: 305ms
788:	learn: 0.0688287	total: 1.13s	remaining: 303ms
789:	learn: 0.0686360	total: 1.14s	remaining: 302ms
790:	learn: 0.0685167	total: 1.14s	remaining: 300ms
791:	learn: 0.0684512	total: 1.14s	remaining: 299ms
792:	learn: 0.0683826	total: 1.14s	remaining: 298ms
793:	learn: 0.0682828	total: 1.14s	remaining: 296ms
794:	learn: 0.0682223	total: 1.14s	remaining: 295ms
795:	learn: 0.0681118	total: 1.14s	remaining: 293ms
796:	learn: 0.0680200	total: 1.15s	remaining: 292ms
797:	learn: 0.0678713	total: 1.15s	remaining: 290ms
798:	learn: 0.0677624	total: 1.15s	remaining: 289ms
799:	learn: 0.0676215	total: 1.15s	remaining: 287ms
800:	learn: 0.0674696	total: 1.15s	remaining: 286ms
801:	learn: 

[08:46:37] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[08:46:37] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[08:46:38] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[08:46:38] WARNING: /home

# 外部验证

## 验证集1

In [16]:
# Load the external validation set 1
V1 = pd.read_csv('top17_validation1.csv', index_col=0)  # Assuming the external validation set is in a CSV file named 'validation1.csv'

# Extract features and target variables
X_V1 = V1.drop('group', axis=1)
Y_V1 = V1['group']
print(X_V1.head())

         ARHGAP24     COL4A1     COL4A2     EPHB1       ETS1      FMNL3  \
C00055  12.155062  15.533658  14.650068  6.971840  14.804589  12.868542   
C00192  12.321864  16.169583  15.639637  7.201770  13.658162  11.211754   
P01067  12.449762  17.001995  16.127677  6.639051  15.105061  13.240256   
P01101  12.831036  15.574966  15.010206  7.514270  13.514730  11.478876   
P01121  12.717077  15.654666  14.949760  7.509732  14.475517  12.318479   

            PARVA     PLXND1     TGFBR2       VAV3      YWHAZ       CCL2  \
C00055  14.334271  13.416993  16.206897  10.046041  15.987720  13.003059   
C00192  14.740641  13.563876  15.577935   9.685287  15.840407  13.212583   
P01067  14.665033  14.271888  16.321053  10.995072  16.565476  14.489713   
P01101  14.296967  13.700864  15.521040  10.104131  15.385263  10.797405   
P01121  14.513079  13.854388  16.357171  10.283273  15.631896  11.263175   

             CAV1       PTEN      PTPRB        TEK       TIE1  
C00055  16.310353  14.459233

In [17]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np


# 创建空的结果表格
results_table = pd.DataFrame(columns=['Model', 'Accuracy_mean', 'Accuracy_std', 'Precision_mean', 'Precision_std',
                                      'Recall_mean', 'Recall_std', 'F1_Score_mean', 'F1_Score_std', 'AUC-ROC_mean', 'AUC-ROC_std'])

# 遍历之前生成的模型列表
for model in models:
    
    # 拟合模型到训练数据
    model.fit(X_train, Y_train)
    
    # 在外部验证集1上进行预测
    Y_V1_pred = model.predict(X_V1)
    
    # 计算评估指标的多次计算结果
    accuracy_scores = np.array([accuracy_score(Y_V1, Y_V1_pred)])
    precision_scores = np.array([precision_score(Y_V1, Y_V1_pred)])
    recall_scores = np.array([recall_score(Y_V1, Y_V1_pred)])
    f1_scores = np.array([f1_score(Y_V1, Y_V1_pred)])
    auc_roc_scores = np.array([roc_auc_score(Y_V1, Y_V1_pred)])
    
    # 计算评估指标的均值和标准差
    accuracy_mean = accuracy_scores.mean()
    accuracy_std = accuracy_scores.std()
    precision_mean = precision_scores.mean()
    precision_std = precision_scores.std()
    recall_mean = recall_scores.mean()
    recall_std = recall_scores.std()
    f1_mean = f1_scores.mean()
    f1_std = f1_scores.std()
    auc_roc_mean = auc_roc_scores.mean()
    auc_roc_std = auc_roc_scores.std()
    
    # 将结果添加到结果表格
    results_table = results_table.append({
        'Model': type(model).__name__,
        'Accuracy_mean': accuracy_mean,
        'Accuracy_std': accuracy_std,
        'Precision_mean': precision_mean,
        'Precision_std': precision_std,
        'Recall_mean': recall_mean,
        'Recall_std': recall_std,
        'F1_Score_mean': f1_mean,
        'F1_Score_std': f1_std,
        'AUC-ROC_mean': auc_roc_mean,
        'AUC-ROC_std': auc_roc_std
    }, ignore_index=True)

# 将结果表格导出为CSV文件
results_table.to_csv('ex_validation1_111.csv', index=False)

Learning rate set to 0.004151
0:	learn: 0.6889941	total: 4.11ms	remaining: 4.1s
1:	learn: 0.6843019	total: 6.31ms	remaining: 3.15s
2:	learn: 0.6812478	total: 7.85ms	remaining: 2.61s
3:	learn: 0.6776370	total: 9.69ms	remaining: 2.41s
4:	learn: 0.6743613	total: 11.1ms	remaining: 2.2s
5:	learn: 0.6699861	total: 12.4ms	remaining: 2.05s
6:	learn: 0.6661978	total: 13.7ms	remaining: 1.94s
7:	learn: 0.6617189	total: 15.2ms	remaining: 1.88s
8:	learn: 0.6580687	total: 16.8ms	remaining: 1.85s
9:	learn: 0.6532210	total: 18.3ms	remaining: 1.82s
10:	learn: 0.6491237	total: 19.9ms	remaining: 1.78s
11:	learn: 0.6461339	total: 22.9ms	remaining: 1.88s
12:	learn: 0.6429769	total: 24.3ms	remaining: 1.85s
13:	learn: 0.6388702	total: 25.6ms	remaining: 1.81s
14:	learn: 0.6344027	total: 27ms	remaining: 1.77s
15:	learn: 0.6305323	total: 28.5ms	remaining: 1.75s
16:	learn: 0.6272315	total: 29.7ms	remaining: 1.72s
17:	learn: 0.6234246	total: 31.1ms	remaining: 1.7s
18:	learn: 0.6196416	total: 32.4ms	remaining: 1.6

240:	learn: 0.2328932	total: 358ms	remaining: 1.13s
241:	learn: 0.2323312	total: 360ms	remaining: 1.13s
242:	learn: 0.2318476	total: 361ms	remaining: 1.12s
243:	learn: 0.2309539	total: 363ms	remaining: 1.12s
244:	learn: 0.2302677	total: 364ms	remaining: 1.12s
245:	learn: 0.2293553	total: 366ms	remaining: 1.12s
246:	learn: 0.2285993	total: 367ms	remaining: 1.12s
247:	learn: 0.2278211	total: 369ms	remaining: 1.12s
248:	learn: 0.2272824	total: 370ms	remaining: 1.12s
249:	learn: 0.2265681	total: 371ms	remaining: 1.11s
250:	learn: 0.2260260	total: 373ms	remaining: 1.11s
251:	learn: 0.2251911	total: 374ms	remaining: 1.11s
252:	learn: 0.2241726	total: 376ms	remaining: 1.11s
253:	learn: 0.2231493	total: 377ms	remaining: 1.11s
254:	learn: 0.2221967	total: 379ms	remaining: 1.11s
255:	learn: 0.2216030	total: 380ms	remaining: 1.1s
256:	learn: 0.2208054	total: 382ms	remaining: 1.1s
257:	learn: 0.2200698	total: 384ms	remaining: 1.1s
258:	learn: 0.2195286	total: 385ms	remaining: 1.1s
259:	learn: 0.21

483:	learn: 0.1222388	total: 724ms	remaining: 772ms
484:	learn: 0.1219147	total: 730ms	remaining: 776ms
485:	learn: 0.1216322	total: 732ms	remaining: 774ms
486:	learn: 0.1214236	total: 733ms	remaining: 772ms
487:	learn: 0.1211169	total: 735ms	remaining: 771ms
488:	learn: 0.1209463	total: 736ms	remaining: 769ms
489:	learn: 0.1206173	total: 737ms	remaining: 768ms
490:	learn: 0.1205050	total: 739ms	remaining: 766ms
491:	learn: 0.1203344	total: 740ms	remaining: 764ms
492:	learn: 0.1200630	total: 742ms	remaining: 763ms
493:	learn: 0.1198311	total: 743ms	remaining: 761ms
494:	learn: 0.1195541	total: 744ms	remaining: 759ms
495:	learn: 0.1193902	total: 746ms	remaining: 758ms
496:	learn: 0.1191340	total: 747ms	remaining: 756ms
497:	learn: 0.1188643	total: 748ms	remaining: 754ms
498:	learn: 0.1187124	total: 750ms	remaining: 753ms
499:	learn: 0.1184856	total: 751ms	remaining: 751ms
500:	learn: 0.1183050	total: 752ms	remaining: 749ms
501:	learn: 0.1180826	total: 754ms	remaining: 748ms
502:	learn: 

745:	learn: 0.0739452	total: 1.1s	remaining: 375ms
746:	learn: 0.0738424	total: 1.1s	remaining: 374ms
747:	learn: 0.0736821	total: 1.1s	remaining: 372ms
748:	learn: 0.0735700	total: 1.1s	remaining: 371ms
749:	learn: 0.0734453	total: 1.11s	remaining: 369ms
750:	learn: 0.0733560	total: 1.11s	remaining: 368ms
751:	learn: 0.0731765	total: 1.11s	remaining: 366ms
752:	learn: 0.0729927	total: 1.11s	remaining: 365ms
753:	learn: 0.0728618	total: 1.11s	remaining: 363ms
754:	learn: 0.0727797	total: 1.11s	remaining: 362ms
755:	learn: 0.0727145	total: 1.11s	remaining: 360ms
756:	learn: 0.0726281	total: 1.12s	remaining: 359ms
757:	learn: 0.0724857	total: 1.12s	remaining: 357ms
758:	learn: 0.0723906	total: 1.12s	remaining: 356ms
759:	learn: 0.0722520	total: 1.12s	remaining: 354ms
760:	learn: 0.0722110	total: 1.12s	remaining: 353ms
761:	learn: 0.0720623	total: 1.13s	remaining: 351ms
762:	learn: 0.0718541	total: 1.13s	remaining: 350ms
763:	learn: 0.0718184	total: 1.13s	remaining: 348ms
764:	learn: 0.07

998:	learn: 0.0507690	total: 1.48s	remaining: 1.48ms
999:	learn: 0.0506812	total: 1.48s	remaining: 0us
[08:46:41] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


## 验证集2

In [18]:
# Load the external validation set 2
V2 = pd.read_csv('top17_validation2.csv', index_col=0)  # Assuming the external validation set is in a CSV file named 'validation1.csv'

# Extract features and target variables
X_V2 = np.log(V2.drop('group', axis=1))
Y_V2 = V2['group']
print(X_V2.head())

            ARHGAP24    COL4A1    COL4A2     EPHB1      ETS1     FMNL3  \
GSM4887610  5.318120  8.049746  7.837554  1.791759  6.839476  5.556828   
GSM4887611  5.023881  7.855157  7.632886  1.609438  6.732211  5.961005   
GSM4887612  5.398163  8.530504  8.200563  2.639057  7.093405  6.398595   
GSM4887613  5.204007  7.737616  7.588324  1.609438  6.593045  5.402677   
GSM4887614  5.817111  8.547528  8.426831  2.197225  7.166266  5.966147   

               PARVA    PLXND1    TGFBR2      VAV3     YWHAZ      CCL2  \
GSM4887610  7.060476  6.405228  8.368693  3.555348  7.443664  3.637586   
GSM4887611  6.862758  5.921578  7.852828  3.610918  7.580189  5.587249   
GSM4887612  7.080868  6.553933  7.852828  3.637586  8.004366  5.837730   
GSM4887613  6.175867  5.743003  7.627057  3.555348  7.379008  4.143135   
GSM4887614  7.166266  7.030857  8.241440  4.174387  7.828038  5.579730   

                CAV1      PTEN     PTPRB       TEK      TIE1  
GSM4887610  7.960324  7.271704  7.684324  6.261

In [19]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np


# 创建空的结果表格
results_table = pd.DataFrame(columns=['Model', 'Accuracy_mean', 'Accuracy_std', 'Precision_mean', 'Precision_std',
                                      'Recall_mean', 'Recall_std', 'F1_Score_mean', 'F1_Score_std', 'AUC-ROC_mean', 'AUC-ROC_std'])

# 遍历之前生成的模型列表
for model in models:
    
    # 拟合模型到训练数据
    model.fit(X_train, Y_train)
    
    # 在外部验证集1上进行预测
    Y_V2_pred = model.predict(X_V2)
    
    # 计算评估指标的多次计算结果
    accuracy_scores = np.array([accuracy_score(Y_V2, Y_V2_pred)])
    precision_scores = np.array([precision_score(Y_V2, Y_V2_pred)])
    recall_scores = np.array([recall_score(Y_V2, Y_V2_pred)])
    f1_scores = np.array([f1_score(Y_V2, Y_V2_pred)])
    auc_roc_scores = np.array([roc_auc_score(Y_V2, Y_V2_pred)])
    
    # 计算评估指标的均值和标准差
    accuracy_mean = accuracy_scores.mean()
    accuracy_std = accuracy_scores.std()
    precision_mean = precision_scores.mean()
    precision_std = precision_scores.std()
    recall_mean = recall_scores.mean()
    recall_std = recall_scores.std()
    f1_mean = f1_scores.mean()
    f1_std = f1_scores.std()
    auc_roc_mean = auc_roc_scores.mean()
    auc_roc_std = auc_roc_scores.std()
    
    # 将结果添加到结果表格
    results_table = results_table.append({
        'Model': type(model).__name__,
        'Accuracy_mean': accuracy_mean,
        'Accuracy_std': accuracy_std,
        'Precision_mean': precision_mean,
        'Precision_std': precision_std,
        'Recall_mean': recall_mean,
        'Recall_std': recall_std,
        'F1_Score_mean': f1_mean,
        'F1_Score_std': f1_std,
        'AUC-ROC_mean': auc_roc_mean,
        'AUC-ROC_std': auc_roc_std
    }, ignore_index=True)

# 将结果表格导出为CSV文件
results_table.to_csv('ex_validation2_111.csv', index=False)

Learning rate set to 0.004151
0:	learn: 0.6889941	total: 3.58ms	remaining: 3.58s
1:	learn: 0.6843019	total: 5.36ms	remaining: 2.67s
2:	learn: 0.6812478	total: 7.01ms	remaining: 2.33s
3:	learn: 0.6776370	total: 8.4ms	remaining: 2.09s
4:	learn: 0.6743613	total: 9.91ms	remaining: 1.97s
5:	learn: 0.6699861	total: 11.4ms	remaining: 1.88s
6:	learn: 0.6661978	total: 12.8ms	remaining: 1.81s
7:	learn: 0.6617189	total: 14.2ms	remaining: 1.76s
8:	learn: 0.6580687	total: 15.6ms	remaining: 1.72s
9:	learn: 0.6532210	total: 16.9ms	remaining: 1.68s
10:	learn: 0.6491237	total: 18.2ms	remaining: 1.64s
11:	learn: 0.6461339	total: 19.6ms	remaining: 1.61s
12:	learn: 0.6429769	total: 20.9ms	remaining: 1.59s
13:	learn: 0.6388702	total: 22.3ms	remaining: 1.57s
14:	learn: 0.6344027	total: 23.6ms	remaining: 1.55s
15:	learn: 0.6305323	total: 24.9ms	remaining: 1.53s
16:	learn: 0.6272315	total: 26.2ms	remaining: 1.51s
17:	learn: 0.6234246	total: 27.5ms	remaining: 1.5s
18:	learn: 0.6196416	total: 29.8ms	remaining: 

253:	learn: 0.2231493	total: 358ms	remaining: 1.05s
254:	learn: 0.2221967	total: 359ms	remaining: 1.05s
255:	learn: 0.2216030	total: 361ms	remaining: 1.05s
256:	learn: 0.2208054	total: 362ms	remaining: 1.05s
257:	learn: 0.2200698	total: 363ms	remaining: 1.04s
258:	learn: 0.2195286	total: 365ms	remaining: 1.04s
259:	learn: 0.2185549	total: 366ms	remaining: 1.04s
260:	learn: 0.2179603	total: 367ms	remaining: 1.04s
261:	learn: 0.2173113	total: 369ms	remaining: 1.04s
262:	learn: 0.2165030	total: 370ms	remaining: 1.04s
263:	learn: 0.2158777	total: 371ms	remaining: 1.03s
264:	learn: 0.2152325	total: 373ms	remaining: 1.03s
265:	learn: 0.2145851	total: 374ms	remaining: 1.03s
266:	learn: 0.2140419	total: 375ms	remaining: 1.03s
267:	learn: 0.2133094	total: 377ms	remaining: 1.03s
268:	learn: 0.2126132	total: 378ms	remaining: 1.03s
269:	learn: 0.2118895	total: 379ms	remaining: 1.02s
270:	learn: 0.2111704	total: 381ms	remaining: 1.02s
271:	learn: 0.2104999	total: 382ms	remaining: 1.02s
272:	learn: 

515:	learn: 0.1145741	total: 732ms	remaining: 686ms
516:	learn: 0.1143039	total: 734ms	remaining: 686ms
517:	learn: 0.1139459	total: 736ms	remaining: 685ms
518:	learn: 0.1136605	total: 738ms	remaining: 684ms
519:	learn: 0.1133892	total: 739ms	remaining: 682ms
520:	learn: 0.1132401	total: 741ms	remaining: 681ms
521:	learn: 0.1129036	total: 742ms	remaining: 680ms
522:	learn: 0.1127152	total: 743ms	remaining: 678ms
523:	learn: 0.1124072	total: 745ms	remaining: 677ms
524:	learn: 0.1120978	total: 746ms	remaining: 675ms
525:	learn: 0.1118805	total: 748ms	remaining: 674ms
526:	learn: 0.1116593	total: 749ms	remaining: 672ms
527:	learn: 0.1113942	total: 750ms	remaining: 671ms
528:	learn: 0.1110580	total: 752ms	remaining: 669ms
529:	learn: 0.1107396	total: 753ms	remaining: 668ms
530:	learn: 0.1105223	total: 754ms	remaining: 666ms
531:	learn: 0.1102204	total: 756ms	remaining: 665ms
532:	learn: 0.1100097	total: 757ms	remaining: 663ms
533:	learn: 0.1098913	total: 759ms	remaining: 662ms
534:	learn: 

768:	learn: 0.0711749	total: 1.11s	remaining: 333ms
769:	learn: 0.0710649	total: 1.11s	remaining: 331ms
770:	learn: 0.0709736	total: 1.11s	remaining: 330ms
771:	learn: 0.0708341	total: 1.11s	remaining: 328ms
772:	learn: 0.0706981	total: 1.11s	remaining: 327ms
773:	learn: 0.0705671	total: 1.11s	remaining: 325ms
774:	learn: 0.0704681	total: 1.11s	remaining: 324ms
775:	learn: 0.0704011	total: 1.12s	remaining: 322ms
776:	learn: 0.0702850	total: 1.12s	remaining: 321ms
777:	learn: 0.0701370	total: 1.12s	remaining: 320ms
778:	learn: 0.0700292	total: 1.12s	remaining: 318ms
779:	learn: 0.0698796	total: 1.12s	remaining: 317ms
780:	learn: 0.0697821	total: 1.12s	remaining: 315ms
781:	learn: 0.0696914	total: 1.13s	remaining: 314ms
782:	learn: 0.0695743	total: 1.13s	remaining: 312ms
783:	learn: 0.0695047	total: 1.13s	remaining: 311ms
784:	learn: 0.0694052	total: 1.13s	remaining: 310ms
785:	learn: 0.0692995	total: 1.13s	remaining: 308ms
786:	learn: 0.0691848	total: 1.13s	remaining: 307ms
787:	learn: 

[08:46:44] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


# 验证集3

In [23]:
# Load the external validation set 2
V3 = pd.read_csv('top17_validation3.csv', index_col=0)  # Assuming the external validation set is in a CSV file named 'validation1.csv'

# Extract features and target variables
X_V3 = np.log(V3.drop('group', axis=1))
Y_V3 = V3['group']
print(X_V3.head())

        ARHGAP24    COL4A1    COL4A2     EPHB1      ETS1     FMNL3     PARVA  \
HCM269  4.015523  8.754774  8.502307  1.610989  4.185802  5.207922  7.082868   
HCM273  4.356200  8.719981  8.632684  1.740382  5.403042  6.614337  7.100778   
HCM282  4.898563  8.753757  8.565665  1.471988  5.483547  5.958949  7.269282   
HCM395  4.737978  8.677337  8.437780  2.272268  5.057213  6.322653  7.062226   
HCM405  5.099195  9.079378  8.851824  3.001963  5.361701  6.754860  6.972306   

          PLXND1    TGFBR2      VAV3     YWHAZ      CCL2      CAV1      PTEN  \
HCM269  5.848907  7.590665  3.570659  5.595317  3.809846  7.229433  7.308616   
HCM273  6.901188  7.712359  3.570659  5.349091  4.115733  7.760910  7.497557   
HCM282  6.956641  7.778325  3.570659  5.700855  4.418737  7.910686  7.843547   
HCM395  6.695639  7.570196  3.344274  5.628678  4.425129  7.514369  7.469831   
HCM405  6.981703  7.672893  3.570659  4.174176  3.845370  7.895466  7.423718   

           PTPRB       TEK      TIE1  

In [24]:


import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np


# 创建空的结果表格
results_table = pd.DataFrame(columns=['Model', 'Accuracy_mean', 'Accuracy_std', 'Precision_mean', 'Precision_std',
                                      'Recall_mean', 'Recall_std', 'F1_Score_mean', 'F1_Score_std', 'AUC-ROC_mean', 'AUC-ROC_std'])

# 遍历之前生成的模型列表
for model in models:
    
    # 拟合模型到训练数据
    model.fit(X_train, Y_train)
    
    # 在外部验证集1上进行预测
    Y_V3_pred = model.predict(X_V3)
    
    # 计算评估指标的多次计算结果
    accuracy_scores = np.array([accuracy_score(Y_V3, Y_V3_pred)])
    precision_scores = np.array([precision_score(Y_V3, Y_V3_pred)])
    recall_scores = np.array([recall_score(Y_V3, Y_V3_pred)])
    f1_scores = np.array([f1_score(Y_V3, Y_V3_pred)])
    auc_roc_scores = np.array([roc_auc_score(Y_V3, Y_V3_pred)])
    
    # 计算评估指标的均值和标准差
    accuracy_mean = accuracy_scores.mean()
    accuracy_std = accuracy_scores.std()
    precision_mean = precision_scores.mean()
    precision_std = precision_scores.std()
    recall_mean = recall_scores.mean()
    recall_std = recall_scores.std()
    f1_mean = f1_scores.mean()
    f1_std = f1_scores.std()
    auc_roc_mean = auc_roc_scores.mean()
    auc_roc_std = auc_roc_scores.std()
    
    # 将结果添加到结果表格
    results_table = results_table.append({
        'Model': type(model).__name__,
        'Accuracy_mean': accuracy_mean,
        'Accuracy_std': accuracy_std,
        'Precision_mean': precision_mean,
        'Precision_std': precision_std,
        'Recall_mean': recall_mean,
        'Recall_std': recall_std,
        'F1_Score_mean': f1_mean,
        'F1_Score_std': f1_std,
        'AUC-ROC_mean': auc_roc_mean,
        'AUC-ROC_std': auc_roc_std
    }, ignore_index=True)

# 将结果表格导出为CSV文件
results_table.to_csv('ex_validation3_111.csv', index=False)

Learning rate set to 0.004151
0:	learn: 0.6889941	total: 3.65ms	remaining: 3.65s
1:	learn: 0.6843019	total: 4.96ms	remaining: 2.48s
2:	learn: 0.6812478	total: 6.56ms	remaining: 2.18s
3:	learn: 0.6776370	total: 8.04ms	remaining: 2s
4:	learn: 0.6743613	total: 9.68ms	remaining: 1.93s
5:	learn: 0.6699861	total: 11ms	remaining: 1.82s
6:	learn: 0.6661978	total: 12.3ms	remaining: 1.74s
7:	learn: 0.6617189	total: 13.5ms	remaining: 1.68s
8:	learn: 0.6580687	total: 15ms	remaining: 1.65s
9:	learn: 0.6532210	total: 16.3ms	remaining: 1.61s
10:	learn: 0.6491237	total: 17.6ms	remaining: 1.58s
11:	learn: 0.6461339	total: 18.8ms	remaining: 1.55s
12:	learn: 0.6429769	total: 20.3ms	remaining: 1.54s
13:	learn: 0.6388702	total: 21.6ms	remaining: 1.52s
14:	learn: 0.6344027	total: 23.2ms	remaining: 1.52s
15:	learn: 0.6305323	total: 24.5ms	remaining: 1.51s
16:	learn: 0.6272315	total: 25.9ms	remaining: 1.5s
17:	learn: 0.6234246	total: 27.3ms	remaining: 1.49s
18:	learn: 0.6196416	total: 28.6ms	remaining: 1.48s


251:	learn: 0.2251911	total: 353ms	remaining: 1.05s
252:	learn: 0.2241726	total: 354ms	remaining: 1.04s
253:	learn: 0.2231493	total: 356ms	remaining: 1.04s
254:	learn: 0.2221967	total: 357ms	remaining: 1.04s
255:	learn: 0.2216030	total: 358ms	remaining: 1.04s
256:	learn: 0.2208054	total: 360ms	remaining: 1.04s
257:	learn: 0.2200698	total: 361ms	remaining: 1.04s
258:	learn: 0.2195286	total: 362ms	remaining: 1.04s
259:	learn: 0.2185549	total: 364ms	remaining: 1.03s
260:	learn: 0.2179603	total: 365ms	remaining: 1.03s
261:	learn: 0.2173113	total: 366ms	remaining: 1.03s
262:	learn: 0.2165030	total: 368ms	remaining: 1.03s
263:	learn: 0.2158777	total: 369ms	remaining: 1.03s
264:	learn: 0.2152325	total: 370ms	remaining: 1.03s
265:	learn: 0.2145851	total: 372ms	remaining: 1.02s
266:	learn: 0.2140419	total: 373ms	remaining: 1.02s
267:	learn: 0.2133094	total: 374ms	remaining: 1.02s
268:	learn: 0.2126132	total: 376ms	remaining: 1.02s
269:	learn: 0.2118895	total: 377ms	remaining: 1.02s
270:	learn: 

513:	learn: 0.1150159	total: 720ms	remaining: 681ms
514:	learn: 0.1147526	total: 721ms	remaining: 679ms
515:	learn: 0.1145741	total: 723ms	remaining: 678ms
516:	learn: 0.1143039	total: 724ms	remaining: 676ms
517:	learn: 0.1139459	total: 725ms	remaining: 675ms
518:	learn: 0.1136605	total: 727ms	remaining: 673ms
519:	learn: 0.1133892	total: 728ms	remaining: 672ms
520:	learn: 0.1132401	total: 729ms	remaining: 670ms
521:	learn: 0.1129036	total: 730ms	remaining: 669ms
522:	learn: 0.1127152	total: 732ms	remaining: 667ms
523:	learn: 0.1124072	total: 733ms	remaining: 666ms
524:	learn: 0.1120978	total: 734ms	remaining: 665ms
525:	learn: 0.1118805	total: 736ms	remaining: 663ms
526:	learn: 0.1116593	total: 737ms	remaining: 662ms
527:	learn: 0.1113942	total: 738ms	remaining: 660ms
528:	learn: 0.1110580	total: 740ms	remaining: 659ms
529:	learn: 0.1107396	total: 741ms	remaining: 657ms
530:	learn: 0.1105223	total: 742ms	remaining: 655ms
531:	learn: 0.1102204	total: 743ms	remaining: 654ms
532:	learn: 

800:	learn: 0.0676403	total: 1.1s	remaining: 273ms
801:	learn: 0.0675173	total: 1.1s	remaining: 271ms
802:	learn: 0.0673714	total: 1.1s	remaining: 270ms
803:	learn: 0.0673074	total: 1.1s	remaining: 269ms
804:	learn: 0.0672524	total: 1.1s	remaining: 267ms
805:	learn: 0.0671528	total: 1.1s	remaining: 266ms
806:	learn: 0.0670323	total: 1.11s	remaining: 265ms
807:	learn: 0.0668855	total: 1.11s	remaining: 263ms
808:	learn: 0.0667666	total: 1.11s	remaining: 262ms
809:	learn: 0.0667105	total: 1.11s	remaining: 260ms
810:	learn: 0.0666260	total: 1.11s	remaining: 259ms
811:	learn: 0.0665718	total: 1.11s	remaining: 258ms
812:	learn: 0.0665136	total: 1.11s	remaining: 256ms
813:	learn: 0.0663862	total: 1.11s	remaining: 255ms
814:	learn: 0.0662523	total: 1.12s	remaining: 253ms
815:	learn: 0.0661599	total: 1.12s	remaining: 252ms
816:	learn: 0.0660026	total: 1.12s	remaining: 251ms
817:	learn: 0.0659203	total: 1.12s	remaining: 249ms
818:	learn: 0.0657766	total: 1.12s	remaining: 248ms
819:	learn: 0.0656

[08:48:47] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
